In [2]:
from blazingsql import BlazingContext
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask_cuda import initialize

In [3]:
enable_tcp_over_ucx = True
enable_nvlink = False
enable_infiniband = False

In [4]:
initialize.initialize(create_cuda_context=True,
                      enable_tcp_over_ucx=enable_tcp_over_ucx,
                      enable_nvlink=enable_nvlink,
                      enable_infiniband=enable_infiniband)
cluster = LocalCUDACluster(protocol="ucx",
                           enable_tcp_over_ucx=enable_tcp_over_ucx,
                           enable_nvlink=enable_nvlink,
                           enable_infiniband=enable_infiniband)

client = Client(cluster)
client

bokeh.server.util - WARNING - Host wildcard '*' will allow connections originating from multiple (or possibly all) hostnames or IPs. Use non-wildcard values to restrict access explicitly


Client Scheduler: ucx://127.0.0.1:39978 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 2 Memory: 50.39 GB


In [5]:
!ifconfig

enp5s0: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
        inet 192.168.50.250  netmask 255.255.255.0  broadcast 192.168.50.255
        inet6 fe80::502c:ff78:1930:67a3  prefixlen 64  scopeid 0x20<link>
        ether 10:7b:44:92:72:4e  txqueuelen 1000  (Ethernet)
        RX packets 53271303  bytes 52959396484 (49.3 GiB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 42048777  bytes 16545634103 (15.4 GiB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0
        device memory 0xfa500000-fa51ffff  

lo: flags=73<UP,LOOPBACK,RUNNING>  mtu 65536
        inet 127.0.0.1  netmask 255.0.0.0
        inet6 ::1  prefixlen 128  scopeid 0x10<host>
        loop  txqueuelen 1000  (Local Loopback)
        RX packets 5155139  bytes 7826957082 (7.2 GiB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 5155139  bytes 7826957082 (7.2 GiB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0

tun0: flags=4305<UP,POINTOPOI

In [6]:
bc = BlazingContext(network_interface="enp5s0", dask_client=client)

starting polling thread
starting listeners
Sending listen start message to workers
started listeners
BlazingContext ready


In [9]:
import cudf
df = cudf.DataFrame({"a": cudf.Series([1, 2, 3, 4]), "b": cudf.Series([2, 3, 4, 5])})

import dask_cudf

ddf = dask_cudf.from_cudf(df, npartitions=2)

In [10]:
!nvidia-smi

Fri Jun 26 13:03:17 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Quadro GV100        Off  | 00000000:42:00.0 Off |                  Off |
| 34%   48C    P2    45W / 250W |   1292MiB / 32508MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Quadro GV100        Off  | 00000000:43:00.0  On |                  Off |
| 42%   55C    P2    42W / 250W |   1644MiB / 32505MiB |     16%      Default |
+-------------------------------+----------------------+----------------------+
        

In [11]:
ddf = ddf.persist()

In [12]:
bc.create_table("test_table", ddf)

In [13]:
bc.sql("select * from test_table").compute()

,a,b
0,1,2
1,2,3
0,1,2
1,2,3


In [ ]:
bc.sql("select sum(a) from test_table group by b").compute()